In [ ]:
import os
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn

In [ ]:
CUR_DIR = os.getcwd()
PROJECT_DIR = os.path.dirname(CUR_DIR)
DATA_DIR = os.path.join(PROJECT_DIR, 'data')
RAW_DATA_DIR = os.path.join(DATA_DIR, 'raw')

In [ ]:
def getFiles(folderPath):
    files_name = []
    for root, dirs, files in os.walk(folderPath):
        for file in files:
            files_name.append(file)

    return files_name

In [ ]:
files_name = getFiles(RAW_DATA_DIR)

In [ ]:
FRANC_TEXT = os.path.join(RAW_DATA_DIR, files_name[0])

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
with open(FRANC_TEXT, 'r') as f:
    text = f.read()

chars = sorted(set(text))
vocab_size = len(chars)
block_size = 64
batch_size = 128
num_embd = 500
num_head = 4
num_layer = 4
dropout = 0.2

In [ ]:
str_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_str = {i:ch for i,ch in enumerate(chars)}

encode = lambda x: [str_to_int[c] for c in x]
decode = lambda x: ''.join([int_to_str[i] for i in x])

In [ ]:
# original 
text

In [ ]:
# encode
text_encode = encode(text)

In [ ]:
# decode
decode(text_encode)

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
data

In [ ]:
data.shape

In [ ]:
len(text)

In [ ]:
def get_batch(data, block_size=8, batch_size=4, gap=1):
    index = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in index])
    y = torch.stack([data[i+gap:i+block_size+gap] for i in index])
    return x.to(device), y.to(device)

In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=0)

In [ ]:
train

In [ ]:
X, y = get_batch(train)

In [ ]:
X

In [ ]:
y

In [ ]:
X[0]

In [ ]:
y[0]

In [ ]:
decode(X[0].tolist())

In [ ]:
decode(y[0].tolist())

In [ ]:
import torch.nn.functional as F

In [ ]:
# one head of self-attention
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(num_embd, head_size, bias=False)
        self.query = nn.Linear(num_embd, head_size, bias=False)
        self.value = nn.Linear(num_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        
        w = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        w = w.masked_fill(self.tril[:T, :T]==0, float('-inf'))
        w = F.softmax(w, dim=-1)
        w = self.dropout(w)

        v = self.value(x)
        out = w @ v
        return out

In [ ]:
# multiple heads of self-attention in parallel
class MultiHeadAttention(nn.Module):
    def __init__(self, num_head, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_head)])
        self.proj = nn.Linear(head_size*num_head, num_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [ ]:
# a simple linear layer followed by non-linearity
class FeedFoward(nn.Module):
    def __init__(self, num_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(num_embd, 4*num_embd),
            nn.ReLU(),
            nn.Linear(4*num_embd, num_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
#
class Block(nn.Module):
    def __init__(self, num_embd, num_head):
        super().__init__()
        head_size = num_embd // num_head
        self.sa = MultiHeadAttention(num_head, head_size) # sa = Self Attention
        self.ffwd = FeedFoward(num_embd)
        self.ln1 = nn.LayerNorm(num_embd)
        self.ln2 = nn.LayerNorm(num_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

In [ ]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size, num_embd, block_size, num_head, num_layer):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, num_embd)
        self.position_embedding_table = nn.Embedding(block_size, num_embd)
        self.blocks = nn.Sequential(*[Block(num_embd, num_head=num_head) for _ in range(num_layer)])
        self.ln_f = nn.LayerNorm(num_embd)
        self.lm_head = nn.Linear(num_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)


    def forward(self, index, targets=None):
        B, T = index.shape
        
        # idx and target are both (B, T) tensor of integers
        tok_embd = self.token_embedding_table(index) # (B, T, C)
        pos_embd = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_embd + pos_embd # (B, T, C)
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_token):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_token):
            # get the prediction
            logits, loss = self.forward(index)

            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)

            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)

            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)

        return index 

In [ ]:
model = GPTLanguageModel(vocab_size, num_embd, block_size, num_head, num_layer)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
gen_chars = decode(m.generate(context, max_new_token=500)[0].tolist())
print(gen_chars)

In [ ]:
learning_rate = 3e-4
max_iters = 10000

In [ ]:
def train_model(model, num_epoch, optimizer, train_data, test_data):
    losses = {}

    model.train()
    for epoch in range(num_epoch):
        xb, yb = get_batch(train_data)

        logits, loss = model.forward(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        #
        if epoch % 1000 == 0:
            # track train loss
            losses['train'] = loss.item()

            # evaluation mode
            model.eval()
            xb, yb = get_batch(test_data)
            logits, loss = model(xb, yb)
            losses['test'] = loss.item()

            print("Epoch %d : Train %.4f, Test %.4f" % (epoch, losses['train'], losses['test']))

In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # sample a batch of data
    xb, yb = get_batch(train)

    # evaluate loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

In [ ]:
train_model(model, max_iters, optimizer, train, test)

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
gen_chars = decode(m.generate(context, max_new_token=500)[0].tolist())
print(gen_chars)